In [1]:
import itk
import nibabel as nib
import numpy as np

from itkwidgets import view

In [2]:
# Dataset number 10 from:
# http://nist.mni.mcgill.ca/?page_id=672
# Rivaz, H., Chen, S, Collins, DL., Automatic Deformable MR-Ultrasound Registration for Image-Guided Neurosurgery, IEEE Trans. Medical Imaging, 2015, 34(2): 366–380
file_name = 'MR.mnc'
image = nib.load('MR.mnc')

In [3]:
image = itk.GetImageFromArray(image.get_data().astype(np.float32))

In [ ]:
view(image)

In [4]:
hessian_filter = itk.HessianRecursiveGaussianImageFilter.New(image)

In [5]:
hessian_filter.SetSigma(1.2)
vesselness_filter = itk.Hessian3DToVesselnessMeasureImageFilter[itk.ctype('float')].New()
vesselness_filter.SetInput(hessian_filter.GetOutput())
vesselness_filter.Update()
#view(vesselness_filter)

In [6]:
objectness = itk.HessianToObjectnessMeasureImageFilter[type(hessian_filter.GetOutput()), type(image)].New()
objectness.SetBrightObject(True)
objectness.SetScaleObjectnessMeasure(True)
objectness.SetAlpha(0.5)
objectness.SetBeta(1.0)
objectness.SetGamma(5.0)

In [7]:
multi_scale_vesselness = itk.MultiScaleHessianBasedMeasureImageFilter.New(image)
multi_scale_vesselness.SetHessianToMeasureFilter(objectness)
multi_scale_vesselness.SetSigmaStepMethodToLogarithmic()
multi_scale_vesselness.SetSigmaMinimum(0.5)
multi_scale_vesselness.SetSigmaMaximum(1.5)
multi_scale_vesselness.SetNumberOfSigmaSteps(6)

In [8]:
thresholder = itk.ThresholdImageFilter.New(multi_scale_vesselness)
thresholder.ThresholdBelow(10.0)
thresholder.Update()
#view(thresholder)

In [11]:
vessel_scale = itk.MultiplyImageFilter.New(thresholder)
vessel_scale.SetConstant(2.0)

vessel_enhanced = itk.AddImageFilter.New(image)
vessel_enhanced.SetInput2(vessel_scale.GetOutput())
vessel_enhanced.Update()
#view(vessel_enhanced)

In [12]:
smoothing = itk.GradientAnisotropicDiffusionImageFilter.New(vessel_enhanced)
smoothing.SetNumberOfIterations(10)

region_growing = itk.ConnectedThresholdImageFilter.New(smoothing)
region_growing.SetLower(80)
region_growing.SetUpper(110)
region_growing.SetReplaceValue(1)
index = itk.Index[3]()
index[0] = 61
index[1] = 61
index[2] = 61
region_growing.SetSeed(index)
region_growing.Update()
#view(region_growing)

In [13]:
tumor = itk.MultiplyImageFilter.New(vessel_enhanced)
tumor.SetInput2(region_growing.GetOutput())

tumor_scale = itk.MultiplyImageFilter.New(tumor)
tumor_scale.SetConstant(-3.0)

tumor_enhanced = itk.AddImageFilter.New(vessel_enhanced)
tumor_enhanced.SetInput2(tumor_scale)

tumor_enhanced.Update()
view(tumor_enhanced)

Viewer(image=<itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkImageF3 *' at 0x7fcf1be11780> >)